In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'../..')

import pandas as pd
import numpy as np

from helper import *
from itertools import combinations
from sklearn import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/usr/local/lib/python3.4/dist-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [ ]:

def add_combinations(X, degrees):
    print("Adding combinations of %d features" % ( degrees ))
    combs = combinations(range(X.columns.size), degrees)
    dt = pd.DataFrame()
    for features_i in combs:
        feature_vals = [ hash(tuple(v)) for v in X.ix[:,features_i].values ]
        feature_name = "comb_" +  '_'.join([str(x) for x in features_i])
        dt[feature_name] = feature_vals
        print('.', end="")
    print("")
    return dt

def prepare(X):
    x2 = add_combinations(X, 2)
    x3 = add_combinations(X, 3)
    X = pd.concat([X, x2, x3], axis=1)
    print("Added combinations: ", X.shape)
    return X

In [4]:
SEED = 42

# loading data

test  = pd.read_csv("./test.csv")
train = pd.read_csv("./train.csv")

features = train.columns[~train.columns.isin(['ACTION', 'MGR_ID'])]
X, y, X_test = train[features], train.ACTION, test[features]

In [59]:
pd.DataFrame([train.corr(method='pearson')['ACTION'], train.corr(method='spearman')['ACTION']])

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
ACTION,1.0,0.000185,-0.005167,-0.013702,0.005179,0.001025,-0.010169,0.003565,0.000502,0.017147
ACTION,1.0,0.001058,-0.032743,-0.052251,0.012340,0.027657,0.016940,0.023019,-0.004045,0.019523


In [4]:
X.corr(method='pearson')

,RESOURCE,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
RESOURCE,1.000000,-0.005016,0.013438,0.030004,0.002936,0.021029,0.031060,0.007733
ROLE_ROLLUP_1,-0.005016,1.000000,0.033358,-0.009548,0.010207,-0.007546,0.029468,-0.024927
ROLE_ROLLUP_2,0.013438,0.033358,1.000000,-0.006056,0.008305,0.018873,0.069558,0.015117
ROLE_DEPTNAME,0.030004,-0.009548,-0.006056,1.000000,-0.006932,-0.002877,0.031669,0.010319
ROLE_TITLE,0.002936,0.010207,0.008305,-0.006932,1.000000,0.170692,-0.012450,0.155920
ROLE_FAMILY_DESC,0.021029,-0.007546,0.018873,-0.002877,0.170692,1.000000,-0.180596,0.092980
ROLE_FAMILY,0.031060,0.029468,0.069558,0.031669,-0.012450,-0.180596,1.000000,-0.148625
ROLE_CODE,0.007733,-0.024927,0.015117,0.010319,0.155920,0.092980,-0.148625,1.000000


In [5]:
X.corr(method='spearman')

,RESOURCE,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
RESOURCE,1.000000,0.016169,0.006049,0.012043,0.000799,0.004104,0.043918,0.006140
ROLE_ROLLUP_1,0.016169,1.000000,0.486794,-0.000881,0.035428,0.096698,0.006081,0.039568
ROLE_ROLLUP_2,0.006049,0.486794,1.000000,0.232726,0.094600,0.086547,0.215989,0.110953
ROLE_DEPTNAME,0.012043,-0.000881,0.232726,1.000000,0.183483,0.108153,0.265308,0.210597
ROLE_TITLE,0.000799,0.035428,0.094600,0.183483,1.000000,0.532456,-0.099125,0.916368
ROLE_FAMILY_DESC,0.004104,0.096698,0.086547,0.108153,0.532456,1.000000,-0.174938,0.511507
ROLE_FAMILY,0.043918,0.006081,0.215989,0.265308,-0.099125,-0.174938,1.000000,-0.065955
ROLE_CODE,0.006140,0.039568,0.110953,0.210597,0.916368,0.511507,-0.065955,1.000000


In [5]:
FEATURES    = X.columns
categorical = X.columns

X_train = X
# X_train = prepare(X)
ranking_0 = feature_importance(X_train, y, chi2=True, f_class=True)

X_enc = one_hot_encode(X_train)
ranking_1 = feature_importance(X_enc, y, chi2=True, f_class=True)

/usr/local/lib/python3.4/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.re

In [24]:
X_train.describe()

,RESOURCE,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
count,32769,32769,32769,32769,32769,32769,32769,32769
unique,7518,128,177,449,343,2358,67,343
top,4675,117961,118300,117878,118321,117906,290919,118322
freq,839,21407,4424,1135,4649,6896,10980,4649


In [12]:
ranking_0.sort_values(by='chi2', ascending=False)

,features,gini,std,stability,chi2,f_classif
4,ROLE_TITLE,0.086398,0.039198,0.918033,1.000000,0.351554
5,ROLE_FAMILY_DESC,0.163603,0.053410,0.213115,0.455358,0.043089
1,ROLE_ROLLUP_1,0.117532,0.046588,1.000000,0.239098,0.638367
7,ROLE_CODE,0.084833,0.042500,1.000000,0.102751,1.000000
6,ROLE_FAMILY,0.105858,0.041642,0.147541,0.016356,0.000740
2,ROLE_ROLLUP_2,0.150627,0.056606,0.393443,0.004760,0.091081
3,ROLE_DEPTNAME,0.142492,0.059966,0.000000,0.002834,0.003454
0,RESOURCE,0.148659,0.050596,0.049180,0.000000,0.000000


In [10]:
ranking_1.groupby(lambda x: feature_group(ranking_1, x), axis=0).mean().sort_values(by='gini', ascending=False)

,gini,std,chi2,f_classif
ROLE_FAMILY,0.000471,0.002799,0.022008,0.022415
ROLE_ROLLUP_2,0.000397,0.002548,0.017600,0.017664
ROLE_ROLLUP_1,0.000373,0.002470,0.015817,0.016428
ROLE_DEPTNAME,0.000296,0.001981,0.016158,0.015972
ROLE_CODE,0.000195,0.001505,0.011005,0.010972
ROLE_TITLE,0.000185,0.001382,0.011005,0.010972
ROLE_FAMILY_DESC,0.000102,0.000780,0.005939,0.005850
RESOURCE,0.000046,0.000512,0.003739,0.003677


In [ ]:

models = [
    ensemble.RandomForestClassifier( n_estimators=100, max_depth=7 ),
 linear_model.LogisticRegression(penalty='l2', C=1),
    linear_model.LogisticRegression(penalty='l1', C=1),
    naive_bayes.MultinomialNB(),
]

for m in models:
    evaluate(X_enc, y, m)
    
evaluate(X_enc, y, ensemble.VotingClassifier(
    estimators=[ ( m.__class__.__name__, m ) for m in models ],
    voting='soft'
))


In [ ]:
ranking_1 = feature_importance(X_enc, y)
ranking_0

In [ ]:

# hyperparameter optimization

params = { "C": np.logspace(-4, 4, 15, base=2) }
p = GridSearchCV(model, param_distributions=params, scoring='roc_auc', n_jobs=-1).fit(X, y)

print("Found best params: ", p)

# predicting

model = LogisticRegression(C=p.best_params_.C)
model = model.train(X, y)
predictions = model.predic_proba(X_test)[:,1]

export = pd.DataFrame({ "id": test.id, "ACTION": predictions })
export.to_csv("./predictions.csv", index=False)

print "Done!"

# evaluation

scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
print "Final score: %f +- %f" % (i, scores.mean(), scores.std()*2)